In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
import os 
from dotenv import load_dotenv

In [72]:
# 웹 브라우져를 실행 
driver = webdriver.Chrome()

In [73]:
## 인스타그램에 요청 
driver.get('https://www.instagram.com')

In [74]:
load_dotenv()

True

In [75]:
driver.implicitly_wait(10)
# 인스타그램에 로그인 
id_element = driver.find_element(
    By.CSS_SELECTOR, 
    'input[name="username"]')
# id_element에 아이디 값(.env에 있는 id)을 입력 
id_element.send_keys(os.getenv('id'))
pass_element = driver.find_element(
    By.CSS_SELECTOR, 
    'input[name="password"]'
)
# pass_element에 패스워드(.env에 있는 password)를 입력
pass_element.send_keys(os.getenv('password'))
# time.sleep(2)
# implicitly_wait() : html, css 정보가 모두 로드가 되면 시간에 무관하게 다음 코드가 실행
# 입력한 시간이 초과될때 로드가 전부 안되어있으면 에러 발생
driver.implicitly_wait(10)
pass_element.send_keys(Keys.ENTER)

In [76]:
# svg 태그 중 aria-label="검색" 인 태그를 선택
driver.implicitly_wait(10)
search_element = driver.find_element(By.CSS_SELECTOR, 
                                     'svg[aria-label="검색"]')
# search_element를 클릭
search_element.click()

In [77]:
driver.implicitly_wait(10)
# 검색창에 input 태그를 선택
search_input = driver.find_element(By.CSS_SELECTOR, 
                                   'input[aria-label="입력 검색"]')
# search_input에 특정 문자열을 입력
search_input.send_keys("구디맛집")

In [78]:
driver.implicitly_wait(10)
## 검색 리스트 전체를 검색
list_element = driver.find_elements(
    By.CSS_SELECTOR, 
    '.x9f619.x78zum5.xdt5ytf.x1iyjqo2.x6ikm8r.x1odjw0f.xh8yej3.xocp1fn .x1i10hfl'
)
print(len(list_element))
# 검색 리스트에서 첫번째를 클릭 
list_element[0].click() 


54


In [ ]:
## 게시글 링크를 모두 찾는다. 
imgs = driver.find_elements(
    By.CLASS_NAME, '_aagw'
)
# 게시글의 첫번째를 클릭 
imgs[0].click()

In [ ]:
# 게시물에서 텍스트를 추출 

# Class가 _a9zc인 태그의 수 
# len(driver.find_elements(By.CLASS_NAME, '_a9zs'))

ids = driver.find_elements(By.CLASS_NAME, '_a9zc')
commets = driver.find_elements(By.CLASS_NAME, '_a9zs')

# 데이터프레임에 대입할 변수를 생성 
data = {
    'ID': [], 
    'Commet' : []
}

# for id in ids:
#     print(id.text)
# for commet in commets:
#     print(commet.text.replace('\n', ' '))

# for i in range(len(ids)):
#     print(ids[i].text)
#     print(commets[i].text.replace("\n", " "))

for id, commet in zip(ids, commets):
    # print(id.text)
    # print(commet.text.replace('\n', ' '))
    data['ID'].append(id.text)
    data['Commet'].append(commet.text.replace('\n', ' '))
data

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
# 다음 게시글로 이동하는 버튼을 선택 
next_element = driver.find_element(By.CSS_SELECTOR, 
                                   '._aaqg ._abl-')
next_element.click()

In [ ]:
## 게시글 링크를 모두 찾는다. 
imgs = driver.find_elements(
    By.CLASS_NAME, '_aagw'
)
# 게시글의 첫번째를 클릭 
imgs[0].click()

# data를 저장할 공간 생성
data = {
    'ID' : [], 
    'Commet' : []
}

driver.implicitly_wait(20)
for i in range(3):
    # 다음 버튼이 존재하지 않으면 ? -> 에러 발생 
    # try 구문을 이용하여 에러 발생시 print로 출력
    try:
        driver.implicitly_wait(20)
        # ids = driver.find_elements(By.CLASS_NAME, '_a9zc')
        # commets = driver.find_elements(By.CLASS_NAME, '_a9zs')
        ids = driver.find_elements(By.CSS_SELECTOR, 
                                   'h2[class="_a9zc"]')
        ids.extend(
            driver.find_elements(By.CSS_SELECTOR, 
                                 'h3[class="_a9zc"]')
        )
        commets = driver.find_elements(By.CSS_SELECTOR, 
                                       'div[class="_a9zs"]')
        print(len(ids))
        print(len(commets))
        for id, commet in zip(ids, commets):
            data['ID'].append(id.text)
            data['Commet'].append(commet.text.replace('\n', ' '))
        next_element = driver.find_element(By.CSS_SELECTOR, 
                                           '._aaqg ._abl-')
        next_element.click()
    except:
        print('다음 버튼이 존재하지 않거나 에러 발생')
# 게시물을 닫아준다. 
close_element = driver.find_element(By.CSS_SELECTOR, 
                                    'svg[aria-label="닫기"]')
close_element.click()
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df.loc[0, 'Commet']

In [ ]:
from konlpy.tag import Okt

In [ ]:
# 문자에서 형태소를 분석하는 클래스
okt = Okt()

In [ ]:
# 문자의 형태를 분석 
nouns = okt.nouns(df.loc[0, 'Commet'])

In [ ]:
nouns

In [ ]:
# nouns에서 글자 수가 1인 데이터를 제외
words = [n for n in nouns if len(n) > 1]

In [ ]:
words

In [ ]:
# case1 (map을 이용)
def return_words(x):
    nouns = okt.nouns(x)
    if nouns:
        words = [n for n in nouns if len(n) > 1]
    else:
        words = ""
    return words
    

df['words'] = df['Commet'].map(return_words)

In [ ]:
df

In [47]:
# case2 (for문 이용)
col_data = []
for i in range(len(df)):
    # print(i)
    nouns = okt.nouns(df.loc[i, 'Commet'])
    if nouns:
        words = [n for n in nouns if len(n) > 1]
    else:
        words = ""
    # print(words)
    col_data.append(words)
df['words2'] = col_data

In [50]:
df.to_csv("test.csv")

In [ ]:
df

In [56]:
import requests

In [57]:
# 이미지를 저장하는 함수를 하나 생성 
def image_save(img_path, save_path, file_name):
    html_data = requests.get(img_path)
    imageFile = open(
        os.path.join(
            save_path, 
            file_name
        ), 
        'wb'
    )
    # 이미지 데이터의 크기 
    chunk_size = 100000000
    for chunk in html_data.iter_content(chunk_size):
        imageFile.write(chunk)
        imageFile.close()
    print('파일 저장 완료')

In [79]:
imgs = driver.find_elements(By.CLASS_NAME, '_aagw')
imgs[0].click()

driver.implicitly_wait(10)

for i in range(3):
    try : 
        driver.implicitly_wait(10)
        # 이미지를 저장 
        img_element = driver.find_element(By.CSS_SELECTOR, 
                                          '._aato ._aagv .x5yr21d.xu96u03.x10l6tqk.x13vifvy.x87ps6o.xh8yej3')
        # img_element = driver.find_element(By.CSS_SELECTOR, 
        #                                   'div[class~="_aato _aagv"]>img[class="x5yr21d xu96u03 x10l6tqk x13vifvy x87ps6o xh8yej3"]')
        # 해당하는 이미지 태그에서 src 속성의 값을 출력
        img_src = img_element.get_attribute('src')
        print(img_src)
        image_save(
            img_src, 
            "./", 
            f"{i}.png"
        )
        # 다음버튼 클릭 
        next_element = driver.find_element(By.CSS_SELECTOR, 
                                           '._aaqg ._abl-')
        next_element.click()
    except:
        print('error')
        # 다음버튼 클릭 
        next_element = driver.find_element(By.CSS_SELECTOR, 
                                           '._aaqg ._abl-')
        next_element.click()

https://scontent.cdninstagram.com/v/t51.29350-15/431751142_3694731790795441_3537009666366567387_n.heic?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE0NDAuc2RyLmYyOTM1MCJ9&_nc_ht=scontent.cdninstagram.com&_nc_cat=101&_nc_ohc=6VU_dVPjp_YAb5Q0-Ko&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzMxODQzMDg1MDk1MzYzMTY5MA%3D%3D.2-ccb7-5&oh=00_AfCCEyGjp3D1vzDWb__pJLpYJSzAejZ3856UipJ0_eb0EQ&oe=6622BE09&_nc_sid=10d13b
파일 저장 완료
https://scontent.cdninstagram.com/v/t51.29350-15/412779690_2152180555126192_4803143512538141057_n.heic?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE4MDAuc2RyLmYyOTM1MCJ9&_nc_ht=scontent.cdninstagram.com&_nc_cat=104&_nc_ohc=hlOIQkllAfUAb76ajsh&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzI2NjA4NjU2NjgwODM2MjQ0Mg%3D%3D.2-ccb7-5&oh=00_AfBNxlFVq9z4vqIv7LrrRL4pmNiF1pV_laRd9WpQOMU8NQ&oe=6622BF17&_nc_sid=10d13b
파일 저장 완료
https://scontent.cdninstagram.com/v/t51.29350-15/330825053_723258659335830_5217489359993013875_n.webp?stp=dst-jpg_e35&efg=eyJ2ZW5jb2

In [1]:
# 외부의 모듈을 로드 
import insta

In [3]:
import importlib
importlib.reload(insta)

<module 'insta' from 'c:\\Users\\moons\\Documents\\GitHub\\ubion_m_5\\python\\240415\\insta.py'>

In [4]:
insta.search_insta("가디맛집")

AttributeError: 'NoneType' object has no attribute 'click'